# Import

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib import gridspec
import seaborn as sns
import numpy as np
import xarray as xr

# Define colors

In [ ]:
colors = sns.color_palette("colorblind")
colors

In [ ]:
flowline_color = list(colors[3]) + [1.]
glacier_color = list(colors[9]) + [.5]
outline_color = [0., 0., 0., 1.]
annotation_color = list(colors[7]) + [1.]

In [ ]:
axis_color = list(colors[7]) + [1.]
color_1 = list(colors[3]) + [1.]
color_2 = list(colors[0]) + [1.]
color_3 = list(colors[4]) + [1.]
color_4 = list(colors[2]) + [1.]

# Get bed heights and surface heights from files

In [ ]:
folder = 'plot_data/'
linear_filename = 'rec_line_cons_equ_bed_hreg1.nc'
cliff_filename = 'rec_clif_cons_equ_bed_hreg1.nc'
random_filename = 'rec_rand_cons_equ_bed_hreg1.nc'
random_widths_filename = 'rec_line_wide_equ_bed_hreg1.nc'

In [ ]:
with xr.open_dataset(folder + linear_filename) as ds:
    linear_x = ds.coords['total_distance'].values
    linear_bed_h = ds['total_true_bed_h'].values
    linear_sfc_h = ds['true_surface_h'].values
    cons_width = ds['true_widths'].values

with xr.open_dataset(folder + cliff_filename) as ds:
    cliff_x = ds.coords['total_distance'].values
    cliff_bed_h = ds['total_true_bed_h'].values
    cliff_sfc_h = ds['true_surface_h'].values
    
with xr.open_dataset(folder + random_filename) as ds:
    random_x = ds.coords['total_distance'].values
    random_bed_h = ds['total_true_bed_h'].values
    random_sfc_h = ds['true_surface_h'].values

with xr.open_dataset(folder + random_widths_filename) as ds:
    random_width = ds['true_widths'].values

top_index = np.int(np.round(100/4))
bottom_index = np.int(100 - top_index)
wide_top_width = np.append(np.ones(top_index) * 1., np.ones(bottom_index) * 0.5) * 100

# Start plot

In [ ]:
centerline_linewidth = 2.5
width_linewidth = 1.5
outline_linewidth = 2.
annotation_linewidth = 0.8
head_marker = 'o'
head_marker_size = 150
tail_marker = 'X'
tail_marker_size = 150
fontsize = 25

## Presentation figure

In [ ]:
figsize=(20,14)
fig = plt.figure(figsize=figsize, facecolor='white')
spec = gridspec.GridSpec(ncols=1, nrows=2,
                         height_ratios=[2, 1])

ax = fig.add_subplot(spec[0])

def plot_bed(x_use, y_bed_h, y_sfc_h, ax, description):
    # bed_h
    plt.plot(x_use, y_bed_h, c=flowline_color, lw=centerline_linewidth, zorder=2)
    # head and tail marker
    plt.plot(x_use[0], y_bed_h[0],
             head_marker,
             markersize=head_marker_size / 10,
             color=flowline_color)
    plt.plot(x_use[-1], y_bed_h[-1],
             tail_marker,
             markersize=tail_marker_size / 10,
             color=flowline_color)
    # glacier polygon
    x_polygon = np.concatenate((x_use, x_use[::-1]))
    y_polygon = np.concatenate((y_bed_h, y_sfc_h[::-1]))
    coord_polygon = np.concatenate((np.expand_dims(x_polygon, axis=1),np.expand_dims(y_polygon, axis=1)), axis=1)
    ax.add_patch(Polygon(coord_polygon,
                         fc =glacier_color,  
                         ec =outline_color,
                         closed=False,
                         lw = outline_linewidth-1,
                         zorder=1))
    
    # add description
    plt.text(x_use[0] + 2,
             y_bed_h[0],
             description,
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize)

# linear bed
plot_bed(linear_x + 1, linear_bed_h, linear_sfc_h, ax, 'linear')

# cliff bed
plot_bed(cliff_x + 5, cliff_bed_h, cliff_sfc_h, ax, 'cliff')

# random bed
plot_bed(random_x + 9, random_bed_h, random_sfc_h, ax, 'random')

# add lines and text for height
plt.plot([linear_x[0] + 1, linear_x[0] + 1 -1],
         [linear_bed_h[0], linear_bed_h[0]],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(linear_x[0] + 1 - 1 - 1,
         linear_bed_h[0],
         '4000 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)
plt.plot([linear_x[-1] + 1, linear_x[-1] + 1 -1],
         [linear_bed_h[-1], linear_bed_h[-1]],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(linear_x[-1] + 1 - 1 - 1,
         linear_bed_h[-1],
         '1500 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)

# add lines and text for distance
plt.plot([linear_x[0] + 1, linear_x[0] + 1],
         [linear_bed_h[0], linear_bed_h[0] - 1 * 250],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(linear_x[0] + 1,
         linear_bed_h[0] - 1 * 250 - 0.5 * 250,
         '0 km',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)
plt.plot([linear_x[-1] + 1, linear_x[-1] + 1],
         [linear_bed_h[-1], linear_bed_h[-1] - 1 * 250],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(linear_x[-1] + 1,
         linear_bed_h[-1] - 1 * 250 - 0.5 * 250,
         '10 km',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)

# add annotation for height of cliff
plt.plot([cliff_x[25] + 5, cliff_x[25] + 5 -0.7],
         [cliff_bed_h[25], cliff_bed_h[25]],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(cliff_x[25] + 5 - 0.7 - 1,
         cliff_bed_h[25],
         '3260 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)

# make rectangular around with annotation
plt.text(16,
         4600,
         'different bed heights',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c='black')
plt.plot([16+3, 19.5, 19.5, -2.5, -2.5, 16-3],
         [4600, 4600, 900, 900, 4600, 4600],
         '-',
         lw=1,
         c='black')

# add coordinatensystem
x_origin = -2
y_origin = 1200
len_x = 1
len_z = 1*500
# add x-axis
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin + len_x, y_origin),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=1),
             zorder=1
             )
plt.text(x_origin + len_x + 0.2,y_origin,'x',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

# add z-axis
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin, y_origin + len_z),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=1),
             zorder=1
             )
plt.text(x_origin ,y_origin + len_z + 0.2*500,'z',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

ax.set_ylim([850, 4800])
ax.set_xlim([-2.6,19.6])
ax.axis('off');

ax = fig.add_subplot(spec[1])

def plot_width(x_use, width_use, ax, description):
    # glacier polygon
    x_polygon = np.concatenate((x_use[:72], x_use[:72][::-1]))
    y_polygon = np.concatenate((width_use[:72] / 2, - width_use[:72][::-1] / 2))
    coord_polygon = np.concatenate((np.expand_dims(x_polygon, axis=1),np.expand_dims(y_polygon, axis=1)), axis=1)
    ax.add_patch(Polygon(coord_polygon,
                         fc =glacier_color,  
                         ec =outline_color,
                         closed=False,
                         lw = outline_linewidth-1,
                         zorder=1))
    # polygon without glacier
    x_polygon = np.concatenate((x_use[71:], x_use[71:][::-1]))
    y_polygon = np.concatenate((width_use[71:] / 2, - width_use[71:][::-1] / 2))
    coord_polygon = np.concatenate((np.expand_dims(x_polygon, axis=1),np.expand_dims(y_polygon, axis=1)), axis=1)
    ax.add_patch(Polygon(coord_polygon,
                         fc ='white',  
                         ec =outline_color,
                         closed=False,
                         lw = outline_linewidth-1,
                         zorder=1))
    # plot flowline
    plt.plot([x_use[0], x_use[-1]], [0, 0], c=flowline_color, lw=centerline_linewidth, zorder=2)
    # head and tail marker
    plt.plot(x_use[0], 0,
             head_marker,
             markersize=head_marker_size / 10,
             color=flowline_color)
    plt.plot(x_use[-1], 0,
             tail_marker,
             markersize=tail_marker_size / 10,
             color=flowline_color)
    # add description
    plt.text(x_use[0] + 2,
             75,
             description,
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize)

# plot constant width
plot_width(linear_x, cons_width, ax, 'constant')

# plot random width
plot_width(linear_x + linear_x[-1] + 1, random_width, ax, 'wide top')

# plot wide top, narrow bottom width
#plot_width(linear_x + linear_x[-1] + 1, wide_top_width, ax, 'wide top, narrow bottom')

# make rectangular around with annotation
plt.text(17.5,
         100,
         'different widths',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c='black')
plt.plot([17.5+2.5, 21.5, 21.5, -2.5, -2.5, 17.5-2.5],
         [100, 100, -80, -80, 100, 100],
         '-',
         lw=1,
         c='black')

# add coordinatensystem
x_origin = -2
y_origin = 0
len_x = 1
len_y = 40
# add x-axis
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin + len_x, y_origin),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=1),
             zorder=1
             )
plt.text(x_origin + len_x + 0.2,y_origin,'x',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

# add y-axis
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin, y_origin + len_y),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=1),
             zorder=1
             )
plt.text(x_origin ,y_origin + len_y + 15,'y',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

# add annotation for 100m width
x_origin = linear_x[93]
y_origin = -cons_width[0]/2
len_y = cons_width[0]
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin, y_origin + len_y),
             arrowprops=dict(arrowstyle='<->',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=centerline_linewidth,
                             linestyle='--'),
             zorder=1
             )
plt.text(x_origin - 1.1 ,y_origin + len_y * 0.7,'100 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

# add annotation for 50m width
x_origin = linear_x[93] + linear_x[-1] + 1
y_origin = -random_width[-1]/2
len_y = random_width[-1]
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin, y_origin + len_y),
             arrowprops=dict(arrowstyle='<->',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=centerline_linewidth,
                             linestyle='--'),
             zorder=1
             )
plt.text(x_origin - 1.1 ,y_origin + len_y * 0.7,'50 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

ax.set_ylim([-85, 105])
ax.set_xlim([-2.7,21.7])
ax.axis('off');
fig.savefig('bed_geometries.pdf',format='pdf',bbox_inches='tight',dpi=300)

## Thesis figure

In [ ]:
figsize=(20,14)
fig = plt.figure(figsize=figsize, facecolor='white')
spec = gridspec.GridSpec(ncols=1, nrows=2,
                         height_ratios=[2, 1])

ax = fig.add_subplot(spec[0])

def plot_bed(x_use, y_bed_h, y_sfc_h, ax, description):
    # bed_h
    plt.plot(x_use, y_bed_h, c=flowline_color, lw=centerline_linewidth, zorder=2)
    # head and tail marker
    plt.plot(x_use[0], y_bed_h[0],
             head_marker,
             markersize=head_marker_size / 10,
             color=flowline_color)
    plt.plot(x_use[-1], y_bed_h[-1],
             tail_marker,
             markersize=tail_marker_size / 10,
             color=flowline_color)
    # glacier polygon
    x_polygon = np.concatenate((x_use, x_use[::-1]))
    y_polygon = np.concatenate((y_bed_h, y_sfc_h[::-1]))
    coord_polygon = np.concatenate((np.expand_dims(x_polygon, axis=1),np.expand_dims(y_polygon, axis=1)), axis=1)
    ax.add_patch(Polygon(coord_polygon,
                         fc =glacier_color,  
                         ec =outline_color,
                         closed=False,
                         lw = outline_linewidth-1,
                         zorder=1))
    
    # add description
    plt.text(x_use[0] + 2,
             y_bed_h[0],
             description,
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize)

# linear bed
plot_bed(linear_x + 1, linear_bed_h, linear_sfc_h, ax, 'linear')

# cliff bed
plot_bed(cliff_x + 5, cliff_bed_h, cliff_sfc_h, ax, 'cliff')

# random bed
plot_bed(random_x + 9, random_bed_h, random_sfc_h, ax, 'random')

# add lines and text for height
plt.plot([linear_x[0] + 1, linear_x[0] + 1 -1],
         [linear_bed_h[0], linear_bed_h[0]],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(linear_x[0] + 1 - 1 - 1,
         linear_bed_h[0],
         '4000 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)
plt.plot([linear_x[-1] + 1, linear_x[-1] + 1 -1],
         [linear_bed_h[-1], linear_bed_h[-1]],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(linear_x[-1] + 1 - 1 - 1,
         linear_bed_h[-1],
         '1500 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)

# add lines and text for distance
plt.plot([linear_x[0] + 1, linear_x[0] + 1],
         [linear_bed_h[0], linear_bed_h[0] - 1 * 250],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(linear_x[0] + 1,
         linear_bed_h[0] - 1 * 250 - 0.5 * 250,
         '0 km',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)
plt.plot([linear_x[-1] + 1, linear_x[-1] + 1],
         [linear_bed_h[-1], linear_bed_h[-1] - 1 * 250],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(linear_x[-1] + 1,
         linear_bed_h[-1] - 1 * 250 - 0.5 * 250,
         '10 km',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)

# add annotation for height of cliff
plt.plot([cliff_x[25] + 5, cliff_x[25] + 5 -0.7],
         [cliff_bed_h[25], cliff_bed_h[25]],
         '--',
         c=annotation_color,
         lw=centerline_linewidth,
         zorder=1)
plt.text(cliff_x[25] + 5 - 0.7 - 1,
         cliff_bed_h[25],
         '3260 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c=annotation_color)

# make rectangular around with annotation
'''
plt.text(16,
         4600,
         'different bed heights',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c='black')
plt.plot([16+3, 19.5, 19.5, -2.5, -2.5, 16-3],
         [4600, 4600, 900, 900, 4600, 4600],
         '-',
         lw=1,
         c='black')
'''

# add (a) annotation
plt.text(-2,
         4600,
         '(a)',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c='black')

# add coordinatensystem
x_origin = -2
y_origin = 1200
len_x = 1
len_z = 1*500
# add x-axis
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin + len_x, y_origin),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=1),
             zorder=1
             )
plt.text(x_origin + len_x + 0.2,y_origin,'x',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

# add z-axis
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin, y_origin + len_z),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=1),
             zorder=1
             )
plt.text(x_origin ,y_origin + len_z + 0.2*500,'z',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

ax.set_ylim([850, 4800])
ax.set_xlim([-2.6,19.6])
ax.axis('off');

ax = fig.add_subplot(spec[1])

def plot_width(x_use, width_use, ax, description):
    # glacier polygon
    x_polygon = np.concatenate((x_use[:72], x_use[:72][::-1]))
    y_polygon = np.concatenate((width_use[:72] / 2, - width_use[:72][::-1] / 2))
    coord_polygon = np.concatenate((np.expand_dims(x_polygon, axis=1),np.expand_dims(y_polygon, axis=1)), axis=1)
    ax.add_patch(Polygon(coord_polygon,
                         fc =glacier_color,  
                         ec =outline_color,
                         closed=False,
                         lw = outline_linewidth-1,
                         zorder=1))
    # polygon without glacier
    x_polygon = np.concatenate((x_use[71:], x_use[71:][::-1]))
    y_polygon = np.concatenate((width_use[71:] / 2, - width_use[71:][::-1] / 2))
    coord_polygon = np.concatenate((np.expand_dims(x_polygon, axis=1),np.expand_dims(y_polygon, axis=1)), axis=1)
    ax.add_patch(Polygon(coord_polygon,
                         fc ='white',  
                         ec =outline_color,
                         closed=False,
                         lw = outline_linewidth-1,
                         zorder=1))
    # plot flowline
    plt.plot([x_use[0], x_use[-1]], [0, 0], c=flowline_color, lw=centerline_linewidth, zorder=2)
    # head and tail marker
    plt.plot(x_use[0], 0,
             head_marker,
             markersize=head_marker_size / 10,
             color=flowline_color)
    plt.plot(x_use[-1], 0,
             tail_marker,
             markersize=tail_marker_size / 10,
             color=flowline_color)
    # add description
    plt.text(x_use[0] + 2,
             75,
             description,
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize)

# plot constant width
plot_width(linear_x, cons_width, ax, 'constant')

# plot random width
plot_width(linear_x + linear_x[-1] + 1, random_width, ax, 'wide top')

# plot wide top, narrow bottom width
#plot_width(linear_x + linear_x[-1] + 1, wide_top_width, ax, 'wide top, narrow bottom')

# make rectangular around with annotation
'''
plt.text(17.5,
         100,
         'different widths',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c='black')
plt.plot([17.5+2.5, 21.5, 21.5, -2.5, -2.5, 17.5-2.5],
         [100, 100, -80, -80, 100, 100],
         '-',
         lw=1,
         c='black')
'''

# add (b) annotation
plt.text(-2,
         100,
         '(b)',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize,
         c='black')

# add coordinatensystem
x_origin = -2
y_origin = 0
len_x = 1
len_y = 40
# add x-axis
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin + len_x, y_origin),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=1),
             zorder=1
             )
plt.text(x_origin + len_x + 0.2,y_origin,'x',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

# add y-axis
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin, y_origin + len_y),
             arrowprops=dict(arrowstyle='<-',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=1),
             zorder=1
             )
plt.text(x_origin ,y_origin + len_y + 15,'y',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

# add annotation for 100m width
x_origin = linear_x[93]
y_origin = -cons_width[0]/2
len_y = cons_width[0]
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin, y_origin + len_y),
             arrowprops=dict(arrowstyle='<->',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=centerline_linewidth,
                             linestyle='--'),
             zorder=1
             )
plt.text(x_origin - 1.1 ,y_origin + len_y * 0.7,'100 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

# add annotation for 50m width
x_origin = linear_x[93] + linear_x[-1] + 1
y_origin = -random_width[-1]/2
len_y = random_width[-1]
plt.annotate(text='',
             xy=(x_origin, y_origin), 
             xytext=(x_origin, y_origin + len_y),
             arrowprops=dict(arrowstyle='<->',
                             mutation_scale=25,
                             color=list(colors[7]) + [1.],
                             lw=centerline_linewidth,
                             linestyle='--'),
             zorder=1
             )
plt.text(x_origin - 1.1 ,y_origin + len_y * 0.7,'50 m',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=fontsize + 2,
         c=list(colors[7]) + [1.])

ax.set_ylim([-85, 105])
ax.set_xlim([-2.7,21.7])
ax.axis('off');
fig.savefig('bed_geometries_thesis.pdf',format='pdf',bbox_inches='tight',dpi=300)